In [ ]:
import pickle
import json
import mmcv
from tqdm import tqdm_notebook as tqdm
from mmdet.datasets import build_dataloader, get_dataset
from pycocotools.coco import COCO
import pycocotools.coco as cocoapi
import pycocotools.mask as MASK

import cv2
from icecream import ic
import matplotlib.pyplot as plt
from mmdet.models import build_detector
from mmcv.runner import load_checkpoint, get_dist_info
import numpy as np
CLASSES = ('tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane','ground-track-field',
               'helicopter', 'harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field',
               'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad')
cfg = mmcv.Config.fromfile("configs/rs_cascade_mask_rcnn_r50_fpn_1x.py")
results = mmcv.load("results.pkl")
result_bbox = mmcv.load("results.pkl.bbox.json", file_format="json")
print(len(result_bbox))
dataset = get_dataset(cfg.data.test)
print((results[0][1][0]))
img_infos = dataset.load_annotations("/home/xfr/mmdetection/data/rscup/annotation/annos_rscup_val.json")
a = json.load(open("/home/xfr/mmdetection/data/rscup/annotation/annos_rscup_train.json"))
print(a["categories"])
m = []
for key in a["categories"]:
    m.append(key['name'])
print(m)
ann = {}
num_cls = set()
last_name = ""
last = ""
for i in tqdm(range(len(results))):
    filename = img_infos[i]['filename']
    img = cv2.imread("./data/rscup/val/" + filename)
    h, w, _ = img.shape
    items = filename.split("_")
    name = items[0]
    if(name == last_name):
        H, W = last
    else:
        H, W = get_original_size(name)
    last_name = name
    last = (H, W)
    if name not in ann:
        ann[name] = {"bbox": [[] for i in range(len(dataset.CLASSES))], "seg": [[] for i in range(len(dataset.CLASSES))]}
    locx = int(float(items[1]))
    locy = int(float(items[2]))
    result = results[i]
    bboxes = result[0]
    segs = result[1]
    for cls in range(len(dataset.CLASSES)):
        bbox_cls = bboxes[cls]
        seg_cls = segs[cls]
        if( len(bbox_cls) > 0):
            for bbox, rle in zip(bbox_cls, seg_cls):
                xmin, ymin, xmax, ymax, score = bbox
#                 cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0,0,255), 2)
#                 cv2.putText(img, str(cls), (int(xmin), int(ymin)), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                xmin += locx
                ymin += locy
                xmax += locx
                ymax += locy
                num_cls.add(cls)
                ann[name]["bbox"][cls].append([xmin, ymin, xmax, ymax, score])
                mask = MASK.decode(rle)

                contours, hierarchy = cv2.findContours((mask).astype(np.uint8), cv2.RETR_TREE,
                                                                 cv2.CHAIN_APPROX_SIMPLE)
                areas = [cv2.contourArea(contour)  for contour in contours]
                index = np.argmax(areas)
                contour = contours[index]
                rect = cv2.minAreaRect(contour)
                box = cv2.boxPoints(rect)
                temp = np.array([box], np.int32)
#                     cv2.polylines(img, temp, 1, (0, 255, 0), 1)
#                     cv2.fillPoly(img, temp, 255)
                box += np.array([locx, locy])
#                     poly = []
#                     for point in box:
#                         poly.append(point[0])
#                         poly.append(point[1])
#                     poly = np.array(poly, np.double)
#                     poly = [poly]
#                     rles = MASK.frPyObjects(poly, H, W)
#                     rle = MASK.merge(rles)
                ann[name]["seg"][cls].append(box)
   # cv2.imwrite("./vis/"+filename, img)
print(len(ann.keys()))
    #coco_eval(result_file, eval_types, dataset.coco)
    # nms
    # toBbox
    # box - mask - rle
    # rles = maskUtils.frPyObjects(segm, h, w)
       #     rle = maskUtils.merge(rles)
   